# HnM Customer Segmentation 

## Data Load

In [1]:
import matplotlib.pyplot as plt
import platform
from matplotlib import font_manager, rc

get_ipython().run_line_magic("matplotlib", "inline")
path = "C:/Windows/Fonts/malgun.ttf"

if platform.system() == "Darwin":
    rc("font", family="AppleGothic")
elif platform.system() == "Windows":
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc("font", family=font_name)

plt.rcParams['axes.unicode_minus'] = False

In [2]:
# Import Dask for processing large datasets

import pandas as pd
from dask import dataframe as dd
import time
import numpy as np

/opt/homebrew/Caskroom/miniconda/base/envs/ds_study/lib/python3.12/site-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [3]:
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import plotly.graph_objects as go
from matplotlib.colors import LinearSegmentedColormap
from matplotlib import colors as mcolors
from scipy.stats import linregress
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.cluster import KMeans
from tabulate import tabulate
from collections import Counter

%matplotlib inline

In [4]:
transactions_train_dd = dd.read_csv('./transactions_train.csv')
# transactions_train_dd = dd.read_csv('/content/drive/MyDrive/머신러닝프로젝트/dataset/transactions_train.csv')

tmp_list = []
# 항상 같은 index 사용, 누구 PC에서도 똑같이 데이터가 들어가게 하기 위한 목적
np.random.seed(42)
# dask dataframe의 partition 당 개수가 조금씩 달라서 여유있게 파티션 개수 +1로 전체 데이터프레임 크기를 나누고, 난수 발생
sampled_idx = np.random.randint(0, int(len(transactions_train_dd) / (transactions_train_dd.npartitions + 1)), size=10000)

transactions_train_df = pd.DataFrame(transactions_train_dd.partitions[0]).loc[sampled_idx].sort_index()

for i in range(1, transactions_train_dd.npartitions):
    transactions_train_df = pd.concat([transactions_train_df, pd.DataFrame(transactions_train_dd.partitions[i]).loc[sampled_idx].sort_index()])

transactions_train_df.reset_index(drop=True, inplace=True)
transactions_train_df.rename(columns={0: 't_dat', 1: 'customer_id', 2: 'article_id', 3: 'price', 4: 'sales_channel_id'}, inplace=True)
transactions_train_df.tail()

,t_dat,customer_id,article_id,price,sales_channel_id
539995,2020-09-22,aaead1fa5369cd911dbec82a14bb279a3ae9523969cc32...,932798001,0.016932,2
539996,2020-09-22,ab9aead5b9d716f61c3071fa23c5528c1f5f72c8730b30...,861464001,0.016932,1
539997,2020-09-22,ac672405aa390e042f9a740a2e029c1ad0b143f3b69185...,904584007,0.025407,2
539998,2020-09-22,acb6effd34b902465c524bf62a170fc973baac58f619af...,909921001,0.025407,2
539999,2020-09-22,ad3663a848dccbddaf28127ccafa0b06c0f65408fc4d7b...,783517002,0.042034,2


In [5]:
articles_df = pd.read_csv('./articles.csv')
# articles_df = pd.read_csv('/content/drive/MyDrive/머신러닝프로젝트/dataset/articles.csv')

In [6]:
transactions_train_df.head()

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000aa7f0dc06cd7174389e76c9e132a67860c5f65f9706...,680912009,0.011847,2
1,2018-09-20,00a95aa4ba8d20f1bad415bab15ca6174a762f7550c819...,598795014,0.01439,1
2,2018-09-20,00cf4505ba84c01f03c47829352054483acb59a9492ed0...,512251004,0.025407,2
3,2018-09-20,0132d21d866697c428e4d6ffb02548fd71966aa88c8f5a...,580469005,0.080492,2
4,2018-09-20,0132d21d866697c428e4d6ffb02548fd71966aa88c8f5a...,608939002,0.040661,2


In [7]:
transactions_train_df.tail()

,t_dat,customer_id,article_id,price,sales_channel_id
539995,2020-09-22,aaead1fa5369cd911dbec82a14bb279a3ae9523969cc32...,932798001,0.016932,2
539996,2020-09-22,ab9aead5b9d716f61c3071fa23c5528c1f5f72c8730b30...,861464001,0.016932,1
539997,2020-09-22,ac672405aa390e042f9a740a2e029c1ad0b143f3b69185...,904584007,0.025407,2
539998,2020-09-22,acb6effd34b902465c524bf62a170fc973baac58f619af...,909921001,0.025407,2
539999,2020-09-22,ad3663a848dccbddaf28127ccafa0b06c0f65408fc4d7b...,783517002,0.042034,2


In [8]:
articles_df.head()

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
2,108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,Off White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
3,110065001,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,9,Black,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
4,110065002,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,10,White,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."


In [9]:
articles_df.tail()

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
105537,953450001,953450,5pk regular Placement1,302,Socks,Socks & Tights,1010014,Placement print,9,Black,...,Socks Bin,F,Menswear,3,Menswear,26,Men Underwear,1021,Socks and Tights,Socks in a fine-knit cotton blend with a small...
105538,953763001,953763,SPORT Malaga tank,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey,A,Ladieswear,1,Ladieswear,2,H&M+,1005,Jersey Fancy,Loose-fitting sports vest top in ribbed fast-d...
105539,956217002,956217,Cartwheel dress,265,Dress,Garment Full body,1010016,Solid,9,Black,...,Jersey,A,Ladieswear,1,Ladieswear,18,Womens Trend,1005,Jersey Fancy,"Short, A-line dress in jersey with a round nec..."
105540,957375001,957375,CLAIRE HAIR CLAW,72,Hair clip,Accessories,1010016,Solid,9,Black,...,Small Accessories,D,Divided,2,Divided,52,Divided Accessories,1019,Accessories,Large plastic hair claw.
105541,959461001,959461,Lounge dress,265,Dress,Garment Full body,1010016,Solid,11,Off White,...,Jersey,A,Ladieswear,1,Ladieswear,18,Womens Trend,1005,Jersey Fancy,Calf-length dress in ribbed jersey made from a...


In [10]:
transactions_train_df[transactions_train_df['t_dat'] < '2020-01-01']

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000aa7f0dc06cd7174389e76c9e132a67860c5f65f9706...,680912009,0.011847,2
1,2018-09-20,00a95aa4ba8d20f1bad415bab15ca6174a762f7550c819...,598795014,0.01439,1
2,2018-09-20,00cf4505ba84c01f03c47829352054483acb59a9492ed0...,512251004,0.025407,2
3,2018-09-20,0132d21d866697c428e4d6ffb02548fd71966aa88c8f5a...,580469005,0.080492,2
4,2018-09-20,0132d21d866697c428e4d6ffb02548fd71966aa88c8f5a...,608939002,0.040661,2
...,...,...,...,...,...
353470,2019-12-31,fc4d46f8201b3fad537016620734bc3832684bd72fc33f...,612719001,0.010153,1
353471,2019-12-31,fc8203d2dc2078a047b2bd5a91134dcae7664b0616fcdd...,767862001,0.016932,2
353472,2019-12-31,fcb0190a51969013d56143d2a87280085ab007955b4e52...,791131001,0.011051,1
353473,2019-12-31,fdfba3b94fbb3c261232b5788480b1d6b1f5d7207957b8...,816598003,0.042356,1


In [11]:
transactions_train_df[transactions_train_df['t_dat'] >= '2020-01-01']

,t_dat,customer_id,article_id,price,sales_channel_id
353475,2020-01-01,00ad61b1bcf6f2d1470d0071b97da5abb67426784fe5a0...,828982004,0.025407,2
353476,2020-01-01,018f84ed6ec3454cd3faadd840733e096ce185352fad4b...,838099001,0.013542,2
353477,2020-01-01,031860c740a5533c60355d62725d826e79db6e12e37118...,822180001,0.042356,2
353478,2020-01-01,038ef4371836fd2730425dca5d0c71e423dfb7a331a314...,821152004,0.024017,2
353479,2020-01-01,0466dce2005b5db9de9ca9805451feef40a66217afdb09...,619764031,0.016932,2
...,...,...,...,...,...
539995,2020-09-22,aaead1fa5369cd911dbec82a14bb279a3ae9523969cc32...,932798001,0.016932,2
539996,2020-09-22,ab9aead5b9d716f61c3071fa23c5528c1f5f72c8730b30...,861464001,0.016932,1
539997,2020-09-22,ac672405aa390e042f9a740a2e029c1ad0b143f3b69185...,904584007,0.025407,2
539998,2020-09-22,acb6effd34b902465c524bf62a170fc973baac58f619af...,909921001,0.025407,2


In [12]:
int(transactions_train_df['t_dat'][0].split('-')[0])

2018

In [13]:
transactions_train_df['year'] = [int(month.split('-')[0]) for month in transactions_train_df['t_dat']]
transactions_train_df['month'] = [int(month.split('-')[1]) for month in transactions_train_df['t_dat']]
transactions_train_df['day'] = [int(month.split('-')[2]) for month in transactions_train_df['t_dat']]
# transactions_train_df[transactions_train_df['t_dat'] == '2019-01-01']
# transactions_train_df[transactions_train_df['t_dat'] == '2019-12-31']

In [14]:
transactions_train_df['count'] = 1

In [15]:
transactions_train_df.tail()

,t_dat,customer_id,article_id,price,sales_channel_id,year,month,day,count
539995,2020-09-22,aaead1fa5369cd911dbec82a14bb279a3ae9523969cc32...,932798001,0.016932,2,2020,9,22,1
539996,2020-09-22,ab9aead5b9d716f61c3071fa23c5528c1f5f72c8730b30...,861464001,0.016932,1,2020,9,22,1
539997,2020-09-22,ac672405aa390e042f9a740a2e029c1ad0b143f3b69185...,904584007,0.025407,2,2020,9,22,1
539998,2020-09-22,acb6effd34b902465c524bf62a170fc973baac58f619af...,909921001,0.025407,2,2020,9,22,1
539999,2020-09-22,ad3663a848dccbddaf28127ccafa0b06c0f65408fc4d7b...,783517002,0.042034,2,2020,9,22,1


In [16]:
merged_df = pd.merge(transactions_train_df, articles_df, on='article_id', how='left')
merged_df

,t_dat,customer_id,article_id,price,sales_channel_id,year,month,day,count,product_code,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,2018-09-20,000aa7f0dc06cd7174389e76c9e132a67860c5f65f9706...,680912009,0.011847,2,2018,9,20,1,680912,...,Tops Fancy Jersey,D,Divided,2,Divided,53,Divided Collection,1005,Jersey Fancy,T-shirt in cotton jersey with a print motif on...
1,2018-09-20,00a95aa4ba8d20f1bad415bab15ca6174a762f7550c819...,598795014,0.01439,1,2018,9,20,1,598795,...,Tops Fancy Jersey,D,Divided,2,Divided,53,Divided Collection,1005,Jersey Fancy,NaN
2,2018-09-20,00cf4505ba84c01f03c47829352054483acb59a9492ed0...,512251004,0.025407,2,2018,9,20,1,512251,...,Basic 1,D,Divided,2,Divided,51,Divided Basics,1002,Jersey Basic,Short dress in lightweight sweatshirt fabric w...
3,2018-09-20,0132d21d866697c428e4d6ffb02548fd71966aa88c8f5a...,580469005,0.080492,2,2018,9,20,1,580469,...,Premium Quality,C,Ladies Accessories,1,Ladieswear,64,Womens Shoes,1020,Shoes,Court shoes with pointed toes. Leather linings...
4,2018-09-20,0132d21d866697c428e4d6ffb02548fd71966aa88c8f5a...,608939002,0.040661,2,2018,9,20,1,608939,...,Trouser,A,Ladieswear,1,Ladieswear,11,Womens Tailoring,1009,Trousers,Ankle-length trousers in woven fabric with ela...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539995,2020-09-22,aaead1fa5369cd911dbec82a14bb279a3ae9523969cc32...,932798001,0.016932,2,2020,9,22,1,932798,...,Tights basic,B,Lingeries/Tights,1,Ladieswear,62,"Womens Nightwear, Socks & Tigh",1021,Socks and Tights,Biker shorts in ribbed jersey designed with th...
539996,2020-09-22,ab9aead5b9d716f61c3071fa23c5528c1f5f72c8730b30...,861464001,0.016932,1,2020,9,22,1,861464,...,Baby Girl Knitwear,G,Baby Sizes 50-98,4,Baby/Children,40,Baby Girl,1003,Knitwear,"Cardigan in soft, fine-knit cotton with a poin..."
539997,2020-09-22,ac672405aa390e042f9a740a2e029c1ad0b143f3b69185...,904584007,0.025407,2,2020,9,22,1,904584,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,"Top in soft faux shearling with a high collar,..."
539998,2020-09-22,acb6effd34b902465c524bf62a170fc973baac58f619af...,909921001,0.025407,2,2020,9,22,1,909921,...,Loungewear,A,Ladieswear,1,Ladieswear,62,"Womens Nightwear, Socks & Tigh",1017,"Under-, Nightwear","Long, oversized T-shirt in soft, organic cotto..."


In [17]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 540000 entries, 0 to 539999
Data columns (total 33 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   t_dat                         540000 non-null  object
 1   customer_id                   540000 non-null  object
 2   article_id                    540000 non-null  object
 3   price                         540000 non-null  object
 4   sales_channel_id              540000 non-null  object
 5   year                          540000 non-null  int64 
 6   month                         540000 non-null  int64 
 7   day                           540000 non-null  int64 
 8   count                         540000 non-null  int64 
 9   product_code                  540000 non-null  int64 
 10  prod_name                     540000 non-null  object
 11  product_type_no               540000 non-null  int64 
 12  product_type_name             540000 non-null  object
 13 

In [18]:
merged_df.dropna(axis=0, inplace=True)
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 538075 entries, 0 to 539999
Data columns (total 33 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   t_dat                         538075 non-null  object
 1   customer_id                   538075 non-null  object
 2   article_id                    538075 non-null  object
 3   price                         538075 non-null  object
 4   sales_channel_id              538075 non-null  object
 5   year                          538075 non-null  int64 
 6   month                         538075 non-null  int64 
 7   day                           538075 non-null  int64 
 8   count                         538075 non-null  int64 
 9   product_code                  538075 non-null  int64 
 10  prod_name                     538075 non-null  object
 11  product_type_no               538075 non-null  int64 
 12  product_type_name             538075 non-null  object
 13  prod

In [ ]:
#merged_df.to_csv('./merged_df.csv', ',', encoding='utf-8')

In [20]:
merged_df.columns # 33 columns

Index(['t_dat', 'customer_id', 'article_id', 'price', 'sales_channel_id',
       'year', 'month', 'day', 'count', 'product_code', 'prod_name',
       'product_type_no', 'product_type_name', 'product_group_name',
       'graphical_appearance_no', 'graphical_appearance_name',
       'colour_group_code', 'colour_group_name', 'perceived_colour_value_id',
       'perceived_colour_value_name', 'perceived_colour_master_id',
       'perceived_colour_master_name', 'department_no', 'department_name',
       'index_code', 'index_name', 'index_group_no', 'index_group_name',
       'section_no', 'section_name', 'garment_group_no', 'garment_group_name',
       'detail_desc'],
      dtype='object')

In [21]:
merged_df.drop(['department_no', 'department_name', 'index_code', 'index_name','colour_group_code', 'colour_group_name', 'perceived_colour_value_id',
       'perceived_colour_value_name'], axis=1, inplace=True)
merged_df.head(3)

,t_dat,customer_id,article_id,price,sales_channel_id,year,month,day,count,product_code,...,graphical_appearance_name,perceived_colour_master_id,perceived_colour_master_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,2018-09-20,000aa7f0dc06cd7174389e76c9e132a67860c5f65f9706...,680912009,0.011847,2,2018,9,20,1,680912,...,Placement print,9,White,2,Divided,53,Divided Collection,1005,Jersey Fancy,T-shirt in cotton jersey with a print motif on...
2,2018-09-20,00cf4505ba84c01f03c47829352054483acb59a9492ed0...,512251004,0.025407,2,2018,9,20,1,512251,...,Solid,3,Orange,2,Divided,51,Divided Basics,1002,Jersey Basic,Short dress in lightweight sweatshirt fabric w...
3,2018-09-20,0132d21d866697c428e4d6ffb02548fd71966aa88c8f5a...,580469005,0.080492,2,2018,9,20,1,580469,...,Solid,5,Black,1,Ladieswear,64,Womens Shoes,1020,Shoes,Court shoes with pointed toes. Leather linings...


In [22]:
merged_df.columns ## 25 columns

Index(['t_dat', 'customer_id', 'article_id', 'price', 'sales_channel_id',
       'year', 'month', 'day', 'count', 'product_code', 'prod_name',
       'product_type_no', 'product_type_name', 'product_group_name',
       'graphical_appearance_no', 'graphical_appearance_name',
       'perceived_colour_master_id', 'perceived_colour_master_name',
       'index_group_no', 'index_group_name', 'section_no', 'section_name',
       'garment_group_no', 'garment_group_name', 'detail_desc'],
      dtype='object')

In [23]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 538075 entries, 0 to 539999
Data columns (total 25 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   t_dat                         538075 non-null  object
 1   customer_id                   538075 non-null  object
 2   article_id                    538075 non-null  object
 3   price                         538075 non-null  object
 4   sales_channel_id              538075 non-null  object
 5   year                          538075 non-null  int64 
 6   month                         538075 non-null  int64 
 7   day                           538075 non-null  int64 
 8   count                         538075 non-null  int64 
 9   product_code                  538075 non-null  int64 
 10  prod_name                     538075 non-null  object
 11  product_type_no               538075 non-null  int64 
 12  product_type_name             538075 non-null  object
 13  prod

In [24]:
merged_df.describe()

,year,month,day,count,product_code,product_type_no,graphical_appearance_no,perceived_colour_master_id,index_group_no,section_no,garment_group_no
count,538075.000000,538075.000000,538075.000000,538075.0,538075.000000,538075.000000,5.380750e+05,538075.000000,538075.000000,538075.000000,538075.000000
mean,2019.207952,6.509966,16.237201,1.0,696609.668096,246.444380,1.009701e+06,7.587862,2.431176,36.467496,1010.704608
std,0.664443,3.272567,8.940440,0.0,133440.710934,67.583416,1.779089e+04,5.055579,4.819260,23.047269,6.541174
min,2018.000000,1.000000,1.000000,1.0,108775.000000,-1.000000,-1.000000e+00,-1.000000,1.000000,2.000000,1001.000000
25%,2019.000000,4.000000,8.000000,1.0,633130.000000,253.000000,1.010012e+06,5.000000,1.000000,15.000000,1005.000000
50%,2019.000000,6.000000,17.000000,1.0,714790.000000,265.000000,1.010016e+06,5.000000,1.000000,47.000000,1010.000000
75%,2020.000000,9.000000,24.000000,1.0,787216.000000,273.000000,1.010016e+06,11.000000,2.000000,60.000000,1017.000000
max,2020.000000,12.000000,31.000000,1.0,956217.000000,761.000000,1.010029e+06,20.000000,26.000000,97.000000,1025.000000


In [25]:
merged_df.shape[0]

538075

## Feature Engineering

## RFM Features

### 1) Regency (R)

In [26]:
customer_data = merged_df.groupby('customer_id')['t_dat'].max().reset_index()
customer_data

,customer_id,t_dat
0,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,2019-05-07
1,00007d2de826758b65a93dd24ce629ed66842531df6699...,2019-06-23
2,00009d946eec3ea54add5ba56d5210ea898def4b46c685...,2019-11-28
3,0000b95f630aaa9313028ce9c41154bb95ac7afa34f55b...,2019-03-02
4,0000c97821eb48d0e590fd309133f0a6c08f7750f64ccc...,2020-07-22
...,...,...
331984,fffef3b6b73545df065b521e19f64bf6fe93bfd450ab20...,2020-08-08
331985,ffff12aa623c69eae8959d673f1f12ad0194ad760d77fd...,2020-06-26
331986,ffff61677073258d461e043cc9ed4ed97be5617a920640...,2019-04-28
331987,ffffcd5046a6143d29a04fb8c424ce494a76e5cdf4fab5...,2020-08-08


In [27]:
most_recent_data = merged_df['t_dat'].max()
most_recent_data

'2020-09-22'

In [28]:
customer_data['t_dat'] = pd.to_datetime(customer_data['t_dat'])
most_recent_data = pd.to_datetime(most_recent_data)

In [29]:
customer_data['Days_Since_Last_Purchase'] = (most_recent_data - customer_data['t_dat']).dt.days
customer_data

,customer_id,t_dat,Days_Since_Last_Purchase
0,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,2019-05-07,504
1,00007d2de826758b65a93dd24ce629ed66842531df6699...,2019-06-23,457
2,00009d946eec3ea54add5ba56d5210ea898def4b46c685...,2019-11-28,299
3,0000b95f630aaa9313028ce9c41154bb95ac7afa34f55b...,2019-03-02,570
4,0000c97821eb48d0e590fd309133f0a6c08f7750f64ccc...,2020-07-22,62
...,...,...,...
331984,fffef3b6b73545df065b521e19f64bf6fe93bfd450ab20...,2020-08-08,45
331985,ffff12aa623c69eae8959d673f1f12ad0194ad760d77fd...,2020-06-26,88
331986,ffff61677073258d461e043cc9ed4ed97be5617a920640...,2019-04-28,513
331987,ffffcd5046a6143d29a04fb8c424ce494a76e5cdf4fab5...,2020-08-08,45


In [30]:
customer_data.drop(columns=['t_dat'], inplace=True)

In [31]:
customer_data.head()

,customer_id,Days_Since_Last_Purchase
0,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,504
1,00007d2de826758b65a93dd24ce629ed66842531df6699...,457
2,00009d946eec3ea54add5ba56d5210ea898def4b46c685...,299
3,0000b95f630aaa9313028ce9c41154bb95ac7afa34f55b...,570
4,0000c97821eb48d0e590fd309133f0a6c08f7750f64ccc...,62


### 2) Frequency (F)

In [32]:
#거래건수 집계를 위해 customer_id와 t_dat를 이용
total_transactions = merged_df.groupby('customer_id')['t_dat'].nunique().reset_index()
total_transactions.rename(columns={'t_dat':'Total_Transactions'}, inplace=True)
total_transactions

,customer_id,Total_Transactions
0,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,2
1,00007d2de826758b65a93dd24ce629ed66842531df6699...,1
2,00009d946eec3ea54add5ba56d5210ea898def4b46c685...,3
3,0000b95f630aaa9313028ce9c41154bb95ac7afa34f55b...,1
4,0000c97821eb48d0e590fd309133f0a6c08f7750f64ccc...,1
...,...,...
331984,fffef3b6b73545df065b521e19f64bf6fe93bfd450ab20...,2
331985,ffff12aa623c69eae8959d673f1f12ad0194ad760d77fd...,1
331986,ffff61677073258d461e043cc9ed4ed97be5617a920640...,1
331987,ffffcd5046a6143d29a04fb8c424ce494a76e5cdf4fab5...,3


In [33]:
total_products_purchased = merged_df.groupby('customer_id')['count'].sum().reset_index()
total_products_purchased.rename(columns={'count':'Total_Products_Purchased'}, inplace=True)
total_products_purchased

,customer_id,Total_Products_Purchased
0,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,2
1,00007d2de826758b65a93dd24ce629ed66842531df6699...,1
2,00009d946eec3ea54add5ba56d5210ea898def4b46c685...,3
3,0000b95f630aaa9313028ce9c41154bb95ac7afa34f55b...,1
4,0000c97821eb48d0e590fd309133f0a6c08f7750f64ccc...,1
...,...,...
331984,fffef3b6b73545df065b521e19f64bf6fe93bfd450ab20...,2
331985,ffff12aa623c69eae8959d673f1f12ad0194ad760d77fd...,1
331986,ffff61677073258d461e043cc9ed4ed97be5617a920640...,1
331987,ffffcd5046a6143d29a04fb8c424ce494a76e5cdf4fab5...,3


In [34]:
customer_data = pd.merge(customer_data, total_transactions, on='customer_id')
customer_data = pd.merge(customer_data, total_products_purchased, on='customer_id')
customer_data.head()

,customer_id,Days_Since_Last_Purchase,Total_Transactions,Total_Products_Purchased
0,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,504,2,2
1,00007d2de826758b65a93dd24ce629ed66842531df6699...,457,1,1
2,00009d946eec3ea54add5ba56d5210ea898def4b46c685...,299,3,3
3,0000b95f630aaa9313028ce9c41154bb95ac7afa34f55b...,570,1,1
4,0000c97821eb48d0e590fd309133f0a6c08f7750f64ccc...,62,1,1


### 3) Monetary (M)

In [35]:
total_spend = merged_df.groupby('customer_id')['price'].sum().reset_index()
total_spend.rename(columns={'price':'Total_Spend'}, inplace=True)
total_spend

,customer_id,Total_Spend
0,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0.101661
1,00007d2de826758b65a93dd24ce629ed66842531df6699...,0.009644
2,00009d946eec3ea54add5ba56d5210ea898def4b46c685...,0.065712
3,0000b95f630aaa9313028ce9c41154bb95ac7afa34f55b...,0.033881
4,0000c97821eb48d0e590fd309133f0a6c08f7750f64ccc...,0.008458
...,...,...
331984,fffef3b6b73545df065b521e19f64bf6fe93bfd450ab20...,0.067169
331985,ffff12aa623c69eae8959d673f1f12ad0194ad760d77fd...,0.06778
331986,ffff61677073258d461e043cc9ed4ed97be5617a920640...,0.021339
331987,ffffcd5046a6143d29a04fb8c424ce494a76e5cdf4fab5...,0.040932


In [36]:
average_transaction_value = total_spend.merge(total_transactions, on='customer_id')
average_transaction_value['Average_Transaction_Value'] = average_transaction_value['Total_Spend'] / total_transactions['Total_Transactions']
average_transaction_value

,customer_id,Total_Spend,Total_Transactions,Average_Transaction_Value
0,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0.101661,2,0.050831
1,00007d2de826758b65a93dd24ce629ed66842531df6699...,0.009644,1,0.009644
2,00009d946eec3ea54add5ba56d5210ea898def4b46c685...,0.065712,3,0.021904
3,0000b95f630aaa9313028ce9c41154bb95ac7afa34f55b...,0.033881,1,0.033881
4,0000c97821eb48d0e590fd309133f0a6c08f7750f64ccc...,0.008458,1,0.008458
...,...,...,...,...
331984,fffef3b6b73545df065b521e19f64bf6fe93bfd450ab20...,0.067169,2,0.033585
331985,ffff12aa623c69eae8959d673f1f12ad0194ad760d77fd...,0.06778,1,0.06778
331986,ffff61677073258d461e043cc9ed4ed97be5617a920640...,0.021339,1,0.021339
331987,ffffcd5046a6143d29a04fb8c424ce494a76e5cdf4fab5...,0.040932,3,0.013644


In [37]:
customer_data = pd.merge(customer_data, total_spend, on='customer_id')
customer_data = pd.merge(customer_data, average_transaction_value[['customer_id', 'Average_Transaction_Value']], on='customer_id')
customer_data.head()

,customer_id,Days_Since_Last_Purchase,Total_Transactions,Total_Products_Purchased,Total_Spend,Average_Transaction_Value
0,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,504,2,2,0.101661,0.050831
1,00007d2de826758b65a93dd24ce629ed66842531df6699...,457,1,1,0.009644,0.009644
2,00009d946eec3ea54add5ba56d5210ea898def4b46c685...,299,3,3,0.065712,0.021904
3,0000b95f630aaa9313028ce9c41154bb95ac7afa34f55b...,570,1,1,0.033881,0.033881
4,0000c97821eb48d0e590fd309133f0a6c08f7750f64ccc...,62,1,1,0.008458,0.008458


## Product Diversity

In [38]:
unique_products_purchased = merged_df.groupby('customer_id')['article_id'].nunique().reset_index()
unique_products_purchased.rename(columns={'article_id':'Unique_Products_Purchased'}, inplace=True)
unique_products_purchased

,customer_id,Unique_Products_Purchased
0,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,2
1,00007d2de826758b65a93dd24ce629ed66842531df6699...,1
2,00009d946eec3ea54add5ba56d5210ea898def4b46c685...,3
3,0000b95f630aaa9313028ce9c41154bb95ac7afa34f55b...,1
4,0000c97821eb48d0e590fd309133f0a6c08f7750f64ccc...,1
...,...,...
331984,fffef3b6b73545df065b521e19f64bf6fe93bfd450ab20...,2
331985,ffff12aa623c69eae8959d673f1f12ad0194ad760d77fd...,1
331986,ffff61677073258d461e043cc9ed4ed97be5617a920640...,1
331987,ffffcd5046a6143d29a04fb8c424ce494a76e5cdf4fab5...,3


In [39]:
customer_data = pd.merge(customer_data, unique_products_purchased, on='customer_id')
customer_data.head()

,customer_id,Days_Since_Last_Purchase,Total_Transactions,Total_Products_Purchased,Total_Spend,Average_Transaction_Value,Unique_Products_Purchased
0,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,504,2,2,0.101661,0.050831,2
1,00007d2de826758b65a93dd24ce629ed66842531df6699...,457,1,1,0.009644,0.009644,1
2,00009d946eec3ea54add5ba56d5210ea898def4b46c685...,299,3,3,0.065712,0.021904,3
3,0000b95f630aaa9313028ce9c41154bb95ac7afa34f55b...,570,1,1,0.033881,0.033881,1
4,0000c97821eb48d0e590fd309133f0a6c08f7750f64ccc...,62,1,1,0.008458,0.008458,1


## Behavioral Features

* 우리 데이터에는 Hour 정보는 없어서 제외

In [40]:
merged_df['t_dat'] = pd.to_datetime(merged_df['t_dat'])
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 538075 entries, 0 to 539999
Data columns (total 25 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   t_dat                         538075 non-null  datetime64[ns]
 1   customer_id                   538075 non-null  object        
 2   article_id                    538075 non-null  object        
 3   price                         538075 non-null  object        
 4   sales_channel_id              538075 non-null  object        
 5   year                          538075 non-null  int64         
 6   month                         538075 non-null  int64         
 7   day                           538075 non-null  int64         
 8   count                         538075 non-null  int64         
 9   product_code                  538075 non-null  int64         
 10  prod_name                     538075 non-null  object        
 11  product_type_no   

In [41]:
merged_df['Day_Of_Week'] = merged_df['t_dat'].dt.dayofweek
merged_df.head()

,t_dat,customer_id,article_id,price,sales_channel_id,year,month,day,count,product_code,...,perceived_colour_master_id,perceived_colour_master_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc,Day_Of_Week
0,2018-09-20,000aa7f0dc06cd7174389e76c9e132a67860c5f65f9706...,680912009,0.011847,2,2018,9,20,1,680912,...,9,White,2,Divided,53,Divided Collection,1005,Jersey Fancy,T-shirt in cotton jersey with a print motif on...,3
2,2018-09-20,00cf4505ba84c01f03c47829352054483acb59a9492ed0...,512251004,0.025407,2,2018,9,20,1,512251,...,3,Orange,2,Divided,51,Divided Basics,1002,Jersey Basic,Short dress in lightweight sweatshirt fabric w...,3
3,2018-09-20,0132d21d866697c428e4d6ffb02548fd71966aa88c8f5a...,580469005,0.080492,2,2018,9,20,1,580469,...,5,Black,1,Ladieswear,64,Womens Shoes,1020,Shoes,Court shoes with pointed toes. Leather linings...,3
4,2018-09-20,0132d21d866697c428e4d6ffb02548fd71966aa88c8f5a...,608939002,0.040661,2,2018,9,20,1,608939,...,9,White,1,Ladieswear,11,Womens Tailoring,1009,Trousers,Ankle-length trousers in woven fabric with ela...,3
5,2018-09-20,01a9a077a83eacc6fd946a60c4ca2aed16219fcd83d31e...,574120001,0.027102,2,2018,9,20,1,574120,...,9,White,2,Divided,53,Divided Collection,1003,Knitwear,"Short jumper in a soft knit with embroidery, l...",3


In [42]:
days_between_purchase = merged_df.groupby('customer_id')['day'].apply(lambda x: x.diff().dropna()) #'day'가 int타입이어서 diff만으로 계산 가능
days_between_purchase

customer_id                                                             
0000423b00ade91418cceaf3b26c6af3dd342b51fd051eec9c12fb36984420fa  168688   -18.0
00009d946eec3ea54add5ba56d5210ea898def4b46c68570cf0096d962cacc75  46735      3.0
                                                                  328993     6.0
0000f2ea26b7f0a9175f428c8cf7743e9e10e193465ecdfa486d5f3bf019d330  369502     3.0
0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37e011580a479e80aa94  129580     4.0
                                                                            ... 
ffffcd5046a6143d29a04fb8c424ce494a76e5cdf4fab53481233731b5c4f8b7  367382    11.0
                                                                  511570   -16.0
ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1778d0116cffd259264  69684      0.0
                                                                  225587   -19.0
                                                                  485390     0.0
Name: day, Length: 206086, dtype: fl

In [43]:
average_days_between_purchases = days_between_purchase.groupby('customer_id').mean().reset_index()
average_days_between_purchases.rename(columns={'day':'Average_Days_Between_Purchases'}, inplace=True)
average_days_between_purchases

,customer_id,Average_Days_Between_Purchases
0,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,-18.000000
1,00009d946eec3ea54add5ba56d5210ea898def4b46c685...,4.500000
2,0000f2ea26b7f0a9175f428c8cf7743e9e10e193465ecd...,3.000000
3,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,1.250000
4,00023e3dd8618bc63ccad995a5ac62e21177338d642d66...,0.000000
...,...,...
113066,fffcd556af797bddc25d6d56600b6e298a19b90624a8ee...,-15.000000
113067,fffe7116f9f68e8ad287fd7b6e33aad4871d7080e77d2d...,-21.000000
113068,fffef3b6b73545df065b521e19f64bf6fe93bfd450ab20...,-21.000000
113069,ffffcd5046a6143d29a04fb8c424ce494a76e5cdf4fab5...,-2.500000


In [44]:
favorite_shopping_day = merged_df.groupby(['customer_id', 'Day_Of_Week'])['count'].size().reset_index(name='Count')
favorint_shopping_day = favorite_shopping_day.loc[favorite_shopping_day.groupby('customer_id')['Count'].idxmax()][['customer_id', 'Day_Of_Week']]
favorint_shopping_day

,customer_id,Day_Of_Week
0,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,1
1,00007d2de826758b65a93dd24ce629ed66842531df6699...,6
3,00009d946eec3ea54add5ba56d5210ea898def4b46c685...,3
4,0000b95f630aaa9313028ce9c41154bb95ac7afa34f55b...,5
5,0000c97821eb48d0e590fd309133f0a6c08f7750f64ccc...,2
...,...,...
463984,fffef3b6b73545df065b521e19f64bf6fe93bfd450ab20...,2
463986,ffff12aa623c69eae8959d673f1f12ad0194ad760d77fd...,4
463987,ffff61677073258d461e043cc9ed4ed97be5617a920640...,6
463988,ffffcd5046a6143d29a04fb8c424ce494a76e5cdf4fab5...,2


In [45]:
customer_data = pd.merge(customer_data, average_days_between_purchases, on='customer_id')
customer_data = pd.merge(customer_data, favorint_shopping_day, on='customer_id')
customer_data.head()

,customer_id,Days_Since_Last_Purchase,Total_Transactions,Total_Products_Purchased,Total_Spend,Average_Transaction_Value,Unique_Products_Purchased,Average_Days_Between_Purchases,Day_Of_Week
0,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,504,2,2,0.101661,0.050831,2,-18.00,1
1,00009d946eec3ea54add5ba56d5210ea898def4b46c685...,299,3,3,0.065712,0.021904,3,4.50,3
2,0000f2ea26b7f0a9175f428c8cf7743e9e10e193465ecd...,238,2,2,0.062898,0.031449,2,3.00,1
3,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,31,5,5,0.160932,0.032186,5,1.25,5
4,00023e3dd8618bc63ccad995a5ac62e21177338d642d66...,96,3,3,0.083,0.027667,3,0.00,1


## Sales Channel Features

* 우리 데이터는 고객 지역정보가 없어서 선호하는 세일즈채널 정보로 작성
* 논의점: 세일즈채널 1과 2의 구매횟수가 동일할 경우엔 어떻게 처리할 것인지


In [46]:
customer_sales_channel = merged_df.groupby('customer_id')['sales_channel_id'].value_counts().unstack().fillna(0)
customer_sales_channel

sales_channel_id,1,2
customer_id,,
0000423b00ade91418cceaf3b26c6af3dd342b51fd051eec9c12fb36984420fa,0.0,2.0
00007d2de826758b65a93dd24ce629ed66842531df6699338c5570910a014cc2,0.0,1.0
00009d946eec3ea54add5ba56d5210ea898def4b46c68570cf0096d962cacc75,0.0,3.0
0000b95f630aaa9313028ce9c41154bb95ac7afa34f55bc8782a1676e35cec3e,0.0,1.0
0000c97821eb48d0e590fd309133f0a6c08f7750f64cccf2ba053e11119093c2,0.0,1.0
...,...,...
fffef3b6b73545df065b521e19f64bf6fe93bfd450ab20e02ce5d1e58a8f700b,0.0,2.0
ffff12aa623c69eae8959d673f1f12ad0194ad760d77fd489cd7c5a4aa9ae240,1.0,0.0
ffff61677073258d461e043cc9ed4ed97be5617a920640ff61024f4619bf41c4,1.0,0.0


In [47]:
customer_main_sales_channel = customer_sales_channel.idxmax(axis=1)
customer_main_sales_channel.name = 'Preferred_Sales_Channel'
customer_main_sales_channel

customer_id
0000423b00ade91418cceaf3b26c6af3dd342b51fd051eec9c12fb36984420fa    2
00007d2de826758b65a93dd24ce629ed66842531df6699338c5570910a014cc2    2
00009d946eec3ea54add5ba56d5210ea898def4b46c68570cf0096d962cacc75    2
0000b95f630aaa9313028ce9c41154bb95ac7afa34f55bc8782a1676e35cec3e    2
0000c97821eb48d0e590fd309133f0a6c08f7750f64cccf2ba053e11119093c2    2
                                                                   ..
fffef3b6b73545df065b521e19f64bf6fe93bfd450ab20e02ce5d1e58a8f700b    2
ffff12aa623c69eae8959d673f1f12ad0194ad760d77fd489cd7c5a4aa9ae240    1
ffff61677073258d461e043cc9ed4ed97be5617a920640ff61024f4619bf41c4    1
ffffcd5046a6143d29a04fb8c424ce494a76e5cdf4fab53481233731b5c4f8b7    1
ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1778d0116cffd259264    1
Name: Preferred_Sales_Channel, Length: 331989, dtype: int64

In [48]:
customer_data = pd.merge(customer_data, customer_main_sales_channel, on='customer_id')
customer_data.head()

,customer_id,Days_Since_Last_Purchase,Total_Transactions,Total_Products_Purchased,Total_Spend,Average_Transaction_Value,Unique_Products_Purchased,Average_Days_Between_Purchases,Day_Of_Week,Preferred_Sales_Channel
0,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,504,2,2,0.101661,0.050831,2,-18.00,1,2
1,00009d946eec3ea54add5ba56d5210ea898def4b46c685...,299,3,3,0.065712,0.021904,3,4.50,3,2
2,0000f2ea26b7f0a9175f428c8cf7743e9e10e193465ecd...,238,2,2,0.062898,0.031449,2,3.00,1,2
3,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,31,5,5,0.160932,0.032186,5,1.25,5,2
4,00023e3dd8618bc63ccad995a5ac62e21177338d642d66...,96,3,3,0.083,0.027667,3,0.00,1,2


## Seasonality & Trend

In [49]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 538075 entries, 0 to 539999
Data columns (total 26 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   t_dat                         538075 non-null  datetime64[ns]
 1   customer_id                   538075 non-null  object        
 2   article_id                    538075 non-null  object        
 3   price                         538075 non-null  object        
 4   sales_channel_id              538075 non-null  object        
 5   year                          538075 non-null  int64         
 6   month                         538075 non-null  int64         
 7   day                           538075 non-null  int64         
 8   count                         538075 non-null  int64         
 9   product_code                  538075 non-null  int64         
 10  prod_name                     538075 non-null  object        
 11  product_type_no   

In [50]:
monthly_spending = merged_df.groupby(['customer_id', 'year', 'month'])['price'].sum().reset_index()
monthly_spending

,customer_id,year,month,price
0,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,2018,9,0.033881
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,2019,5,0.06778
2,00007d2de826758b65a93dd24ce629ed66842531df6699...,2019,6,0.009644
3,00009d946eec3ea54add5ba56d5210ea898def4b46c685...,2018,11,0.041322
4,00009d946eec3ea54add5ba56d5210ea898def4b46c685...,2019,11,0.02439
...,...,...,...,...
492440,ffffcd5046a6143d29a04fb8c424ce494a76e5cdf4fab5...,2020,1,0.024017
492441,ffffcd5046a6143d29a04fb8c424ce494a76e5cdf4fab5...,2020,8,0.008458
492442,ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1...,2018,12,0.027085
492443,ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1...,2019,7,0.020322


In [51]:
seasonal_buying_patterns = monthly_spending.groupby('customer_id')['price'].agg(['mean', 'std']).reset_index()
seasonal_buying_patterns.rename(columns={'mean':'Monthly_Spending_Mean', 'std':'Monthly_Spending_Std'}, inplace=True)
seasonal_buying_patterns

,customer_id,Monthly_Spending_Mean,Monthly_Spending_Std
0,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0.050831,0.023970
1,00007d2de826758b65a93dd24ce629ed66842531df6699...,0.009644,NaN
2,00009d946eec3ea54add5ba56d5210ea898def4b46c685...,0.032856,0.011973
3,0000b95f630aaa9313028ce9c41154bb95ac7afa34f55b...,0.033881,NaN
4,0000c97821eb48d0e590fd309133f0a6c08f7750f64ccc...,0.008458,NaN
...,...,...,...
331984,fffef3b6b73545df065b521e19f64bf6fe93bfd450ab20...,0.033585,0.024389
331985,ffff12aa623c69eae8959d673f1f12ad0194ad760d77fd...,0.06778,NaN
331986,ffff61677073258d461e043cc9ed4ed97be5617a920640...,0.021339,NaN
331987,ffffcd5046a6143d29a04fb8c424ce494a76e5cdf4fab5...,0.013644,0.008983


In [52]:
seasonal_buying_patterns['Monthly_Spending_Std'].fillna(0, inplace=True)

In [53]:
def calculate_trend(spend_data):
  if len(spend_data) > 1:
    x = np.arange(len(spend_data))
    slope, _, _, _, _ = linregress(x, spend_data)
    return slope

  else:
    return 0

In [54]:
#price 형변환 (object -> float)
monthly_spending['price'] = monthly_spending['price'].astype(float)

In [55]:
spending_trends = monthly_spending.groupby('customer_id')['price'].apply(calculate_trend).reset_index()
spending_trends.rename(columns={'price':'Spending_Trend'}, inplace=True)
spending_trends

,customer_id,Spending_Trend
0,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0.033898
1,00007d2de826758b65a93dd24ce629ed66842531df6699...,0.000000
2,00009d946eec3ea54add5ba56d5210ea898def4b46c685...,-0.016932
3,0000b95f630aaa9313028ce9c41154bb95ac7afa34f55b...,0.000000
4,0000c97821eb48d0e590fd309133f0a6c08f7750f64ccc...,0.000000
...,...,...
331984,fffef3b6b73545df065b521e19f64bf6fe93bfd450ab20...,0.034492
331985,ffff12aa623c69eae8959d673f1f12ad0194ad760d77fd...,0.000000
331986,ffff61677073258d461e043cc9ed4ed97be5617a920640...,0.000000
331987,ffffcd5046a6143d29a04fb8c424ce494a76e5cdf4fab5...,0.000000


In [56]:
customer_data = pd.merge(customer_data, seasonal_buying_patterns, on='customer_id')
customer_data = pd.merge(customer_data, spending_trends, on='customer_id')
customer_data.head()

,customer_id,Days_Since_Last_Purchase,Total_Transactions,Total_Products_Purchased,Total_Spend,Average_Transaction_Value,Unique_Products_Purchased,Average_Days_Between_Purchases,Day_Of_Week,Preferred_Sales_Channel,Monthly_Spending_Mean,Monthly_Spending_Std,Spending_Trend
0,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,504,2,2,0.101661,0.050831,2,-18.00,1,2,0.050831,0.023970,0.033898
1,00009d946eec3ea54add5ba56d5210ea898def4b46c685...,299,3,3,0.065712,0.021904,3,4.50,3,2,0.032856,0.011973,-0.016932
2,0000f2ea26b7f0a9175f428c8cf7743e9e10e193465ecd...,238,2,2,0.062898,0.031449,2,3.00,1,2,0.031449,0.003440,-0.004864
3,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,31,5,5,0.160932,0.032186,5,1.25,5,2,0.040233,0.036957,0.026944
4,00023e3dd8618bc63ccad995a5ac62e21177338d642d66...,96,3,3,0.083,0.027667,3,0.00,1,2,0.027667,0.012721,-0.005932


In [57]:
customer_data['customer_id'] = customer_data['customer_id'].astype(str)
customer_data = customer_data.convert_dtypes()
customer_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113071 entries, 0 to 113070
Data columns (total 13 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   customer_id                     113071 non-null  string 
 1   Days_Since_Last_Purchase        113071 non-null  Int64  
 2   Total_Transactions              113071 non-null  Int64  
 3   Total_Products_Purchased        113071 non-null  Int64  
 4   Total_Spend                     113071 non-null  Float64
 5   Average_Transaction_Value       113071 non-null  Float64
 6   Unique_Products_Purchased       113071 non-null  Int64  
 7   Average_Days_Between_Purchases  113071 non-null  Float64
 8   Day_Of_Week                     113071 non-null  Int32  
 9   Preferred_Sales_Channel         113071 non-null  Int64  
 10  Monthly_Spending_Mean           113071 non-null  Float64
 11  Monthly_Spending_Std            113071 non-null  Float64
 12  Spending_Trend  

In [ ]:
customer_data.to_csv('customer_data', ',', encoding='utf-8' )